# Synthesizing data

In [10]:
WIKI_FN = '../wiki/data.jobcensus.rc.wiki.2020-12-2.csv'
# MLA_FN = '../mla/data.jobcensus.mla.2020-11-23.anno.csv'

In [11]:
import pandas as pd,os,sys
from datetime import datetime
import warnings,re
warnings.filterwarnings('ignore')

## Load data

In [12]:
df_wiki0 = pd.read_csv(WIKI_FN)
# df_mla0 = pd.read_csv(MLA_FN)

In [13]:
df_wiki0[:2]

,disc_page,year,page,page_group,section_content,section_links,section_title,IsTT,IsUni,JobType,JobID
0,Rhetoric/Composition 2011,2011,Rhetoric/Composition_2011,Rhetoric/Composition,"<span class=""",NaN,Rhetoric/Composition Jobs 2011,NaN,n,Unknown,Rhetoric/Composition Jobs 2011
1,Rhetoric/Composition 2011,2011,Rhetoric/Composition_2011,Rhetoric/Composition,Link - first posting on page\nSubfield/descrip...,http://www.acu.edu/academics/provost/positions...,"Abilene Christian University (Abilene, TX)",y,y,TT,http://www.acu.edu/academics/provost/positions...


In [14]:
df_wiki0['JobSource']='Wiki'
# df_mla0['JobSource']='JIL'

## Rename cols

In [15]:
df_wiki = df_wiki0[df_wiki0.IsUni=='y'][['section_title','year','section_links','JobType','page_group','JobSource']].fillna('')
df_wiki.columns=['JobTitle','JobYear','JobLink','JobType','JobField','JobSource']
df_wiki

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource
1,"Abilene Christian University (Abilene, TX)",2011,http://www.acu.edu/academics/provost/positions...,TT,Rhetoric/Composition,Wiki
2,Alfred State College (NY),2011,http://www.higheredjobs.com/faculty/details.cf...,TT,Rhetoric/Composition,Wiki
3,American University of Beirut,2011,mailto:as_dean@aub.edu.lb | http://www.aub.edu...,Unknown,Rhetoric/Composition,Wiki
4,"Angelo State University (San Angelo, TX)",2011,http://www.higheredjobs.com/search/details.cfm...,TT,Rhetoric/Composition,Wiki
5,Arizona State University (Advanced Assistant P...,2011,http://www.asu.edu/clas/english | http://www.w...,Unknown,Rhetoric/Composition,Wiki
...,...,...,...,...,...,...
9359,Pacific University (USA:OR) - Visiting Assista...,2020,https://www.pacificu.edu/academics/academic-re...,Non-TT,Rhetoric/Composition,Wiki
9360,University of Tampa (USA:FL) - Professor of In...,2020,,Non-TT,Rhetoric/Composition,Wiki
9361,"Field Instructor, Rhetoric And Composition",2020,,Non-TT,Rhetoric/Composition,Wiki
9362,Weber State University (USA:UT) - Visiting Ass...,2020,https://jobs.weber.edu/postings/9890,Non-TT,Rhetoric/Composition,Wiki


In [17]:
df_wiki[(df_wiki.JobYear == 2020) & (df_wiki.JobField=='Rhetoric/Composition')]

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource
9279,Allen University: Assistant Professor of English,2020,,TT,Rhetoric/Composition,Wiki
9280,"Brock University: Assistant Professor, Busines...",2020,https://brocku.wd3.myworkdayjobs.com/en-US/bro...,TT,Rhetoric/Composition,Wiki
9281,Baruch College: Lecturer - Writing,2020,https://home.cunyfirst.cuny.edu/psp/cnyepprd/G...,Non-TT,Rhetoric/Composition,Wiki
9282,Baruch College: Assistant Professor - Digital ...,2020,https://home.cunyfirst.cuny.edu/psp/cnyepprd/G...,TT,Rhetoric/Composition,Wiki
9283,University of Central Florida: Associate Profe...,2020,https://jobs.ucf.edu/en-us/job/498553/associat...,TT,Rhetoric/Composition,Wiki
...,...,...,...,...,...,...
9359,Pacific University (USA:OR) - Visiting Assista...,2020,https://www.pacificu.edu/academics/academic-re...,Non-TT,Rhetoric/Composition,Wiki
9360,University of Tampa (USA:FL) - Professor of In...,2020,,Non-TT,Rhetoric/Composition,Wiki
9361,"Field Instructor, Rhetoric And Composition",2020,,Non-TT,Rhetoric/Composition,Wiki
9362,Weber State University (USA:UT) - Visiting Ass...,2020,https://jobs.weber.edu/postings/9890,Non-TT,Rhetoric/Composition,Wiki


In [88]:
# df_wiki[(df_wiki.JobYear==2020) & (df_wiki.JobField=='Comparative Literature')]

In [89]:
#df_wiki[df_wiki.JobField=='Ethnic Studies']

### Clean links

In [90]:
# df_mla['JobLink_norm']=df_mla.JobLink.apply(lambda x: '/'.join(x.split('/')[:-2]))
# df_mla.JobLink_norm.value_counts()

In [18]:
def clean_wiki_links(x):
    for l in str(x).split(' | '):
        if 'joblist.mla.org' in l:
            return '/'.join(l.split('/')[:-2])
    return x
df_wiki['JobLink_norm']=df_wiki['JobLink'].apply(clean_wiki_links)
# df_wiki.JobLink_norm.value_counts()

### Find overlap

In [19]:
df_all = df_wiki #.append(df_mla)
df_all

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm
1,"Abilene Christian University (Abilene, TX)",2011,http://www.acu.edu/academics/provost/positions...,TT,Rhetoric/Composition,Wiki,http://www.acu.edu/academics/provost/positions...
2,Alfred State College (NY),2011,http://www.higheredjobs.com/faculty/details.cf...,TT,Rhetoric/Composition,Wiki,http://www.higheredjobs.com/faculty/details.cf...
3,American University of Beirut,2011,mailto:as_dean@aub.edu.lb | http://www.aub.edu...,Unknown,Rhetoric/Composition,Wiki,mailto:as_dean@aub.edu.lb | http://www.aub.edu...
4,"Angelo State University (San Angelo, TX)",2011,http://www.higheredjobs.com/search/details.cfm...,TT,Rhetoric/Composition,Wiki,http://www.higheredjobs.com/search/details.cfm...
5,Arizona State University (Advanced Assistant P...,2011,http://www.asu.edu/clas/english | http://www.w...,Unknown,Rhetoric/Composition,Wiki,http://www.asu.edu/clas/english | http://www.w...
...,...,...,...,...,...,...,...
9359,Pacific University (USA:OR) - Visiting Assista...,2020,https://www.pacificu.edu/academics/academic-re...,Non-TT,Rhetoric/Composition,Wiki,https://www.pacificu.edu/academics/academic-re...
9360,University of Tampa (USA:FL) - Professor of In...,2020,,Non-TT,Rhetoric/Composition,Wiki,
9361,"Field Instructor, Rhetoric And Composition",2020,,Non-TT,Rhetoric/Composition,Wiki,
9362,Weber State University (USA:UT) - Visiting Ass...,2020,https://jobs.weber.edu/postings/9890,Non-TT,Rhetoric/Composition,Wiki,https://jobs.weber.edu/postings/9890


In [20]:
# drop duplicates
df_all=df_all.drop_duplicates(['JobTitle','JobYear','JobLink'])
df_all

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm
1,"Abilene Christian University (Abilene, TX)",2011,http://www.acu.edu/academics/provost/positions...,TT,Rhetoric/Composition,Wiki,http://www.acu.edu/academics/provost/positions...
2,Alfred State College (NY),2011,http://www.higheredjobs.com/faculty/details.cf...,TT,Rhetoric/Composition,Wiki,http://www.higheredjobs.com/faculty/details.cf...
3,American University of Beirut,2011,mailto:as_dean@aub.edu.lb | http://www.aub.edu...,Unknown,Rhetoric/Composition,Wiki,mailto:as_dean@aub.edu.lb | http://www.aub.edu...
4,"Angelo State University (San Angelo, TX)",2011,http://www.higheredjobs.com/search/details.cfm...,TT,Rhetoric/Composition,Wiki,http://www.higheredjobs.com/search/details.cfm...
5,Arizona State University (Advanced Assistant P...,2011,http://www.asu.edu/clas/english | http://www.w...,Unknown,Rhetoric/Composition,Wiki,http://www.asu.edu/clas/english | http://www.w...
...,...,...,...,...,...,...,...
9359,Pacific University (USA:OR) - Visiting Assista...,2020,https://www.pacificu.edu/academics/academic-re...,Non-TT,Rhetoric/Composition,Wiki,https://www.pacificu.edu/academics/academic-re...
9360,University of Tampa (USA:FL) - Professor of In...,2020,,Non-TT,Rhetoric/Composition,Wiki,
9361,"Field Instructor, Rhetoric And Composition",2020,,Non-TT,Rhetoric/Composition,Wiki,
9362,Weber State University (USA:UT) - Visiting Ass...,2020,https://jobs.weber.edu/postings/9890,Non-TT,Rhetoric/Composition,Wiki,https://jobs.weber.edu/postings/9890


In [21]:
jf_list = list(set(df_all.JobField))
jf_list

['Children%27s',
 'TESOL / Applied Linguistics',
 'Drama, Speech, Film %26 New Media',
 'English Literature',
 'Modern British',
 'Medieval',
 'Restoration/18th-Century',
 'Environmental 2014#Christopher Newport University - Lecturer in English .28Environmental Rhetoric .26 Science Writing.29',
 'American Open',
 'African American',
 'American Studies',
 'Comparative Literature',
 'New Media and Digital Humanities',
 'Musicology/Ethnomusicology',
 'New Media %26 Digital Humanities',
 'Theory',
 'Renaissance',
 'Communication and Media Studies',
 'Music Theory/Composition',
 'Generalist',
 'Environmental',
 'Queer/Women%27s/Gender Studies 2014#Texas A.26M - TT Advanced Assistant Professor.2C Gender and Sexuality Studies with an Emphasis in',
 'Comparative',
 'Other Ethnic American',
 '20-21 c. American',
 'Spanish and Portuguese',
 'Creative Writing',
 'Queer/Women%27s/Gender Studies',
 'Rhetoric/Composition',
 'Romanticism/Victorian/19th',
 'Humanities and Social Sciences Postdocs',
 '

In [27]:
print(len(df_all))
df_all = df_all[
            (df_all.JobField != 'CreativeWriting')
            or (df_all.JobField != 'Creative Writing')
            or (df_all.JobField != 'Renaissance')
            or (df_all.JobField != 'Comparative')
            or (df_all.JobField != 'Anglophone')
]
print(len(df_all))

7226


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [22]:
# sanity check: should be 4 -->
df_all[(df_all.JobField=='Rhetoric/Composition') & (df_all.JobYear==2020)]

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm
9279,Allen University: Assistant Professor of English,2020,,TT,Rhetoric/Composition,Wiki,
9280,"Brock University: Assistant Professor, Busines...",2020,https://brocku.wd3.myworkdayjobs.com/en-US/bro...,TT,Rhetoric/Composition,Wiki,https://brocku.wd3.myworkdayjobs.com/en-US/bro...
9281,Baruch College: Lecturer - Writing,2020,https://home.cunyfirst.cuny.edu/psp/cnyepprd/G...,Non-TT,Rhetoric/Composition,Wiki,https://home.cunyfirst.cuny.edu/psp/cnyepprd/G...
9283,University of Central Florida: Associate Profe...,2020,https://jobs.ucf.edu/en-us/job/498553/associat...,TT,Rhetoric/Composition,Wiki,https://jobs.ucf.edu/en-us/job/498553/associat...
9284,Biola University Assistant/Associate Professor...,2020,https://biola.csod.com/ats/careersite/JobDetai...,TT,Rhetoric/Composition,Wiki,https://joblist.mla.org/job-details/1105
...,...,...,...,...,...,...,...
9359,Pacific University (USA:OR) - Visiting Assista...,2020,https://www.pacificu.edu/academics/academic-re...,Non-TT,Rhetoric/Composition,Wiki,https://www.pacificu.edu/academics/academic-re...
9360,University of Tampa (USA:FL) - Professor of In...,2020,,Non-TT,Rhetoric/Composition,Wiki,
9361,"Field Instructor, Rhetoric And Composition",2020,,Non-TT,Rhetoric/Composition,Wiki,
9362,Weber State University (USA:UT) - Visiting Ass...,2020,https://jobs.weber.edu/postings/9890,Non-TT,Rhetoric/Composition,Wiki,https://jobs.weber.edu/postings/9890


## Prune out unknowns

In [23]:
def get_jobtype(title,link=''):
    title=title.lower()
    rgx = re.compile("(\w[\w']*\w|\w)")
    title_words=rgx.findall(title)
    if 'post-doc' in title or 'postdoc' in title: return 'Postdoc'
    if 'visiting' in title: return 'Non-TT'
    if 'professor' in title or 'open-rank' in title: return 'TT'
    if 'fellowship' in title or 'fellow' in title: return 'Postdoc'
    if 'adjunct' in title or 'part-time' in title: return 'Non-TT'
    if 'director' in title: return 'Non-TT'
    if 'instructor' in title or 'freelance' in title: return 'Non-TT'
    if 'vap' in title_words: return 'Non-TT'
    if 'lecturer' in title: return 'Non-TT'
    if link: return get_jobtype_from_mla(link)
    return 'Unknown'

In [24]:
# Fix missing job type errors
df_all['JobType']=[get_jobtype(x) if y=='Unknown' else y
                    for x,y in zip(df_all['JobTitle'],df_all['JobType'])]#.apply(get_jobtype)

In [25]:
df_all[df_all.JobType=='Unknown']

,JobTitle,JobYear,JobLink,JobType,JobField,JobSource,JobLink_norm
3,American University of Beirut,2011,mailto:as_dean@aub.edu.lb | http://www.aub.edu...,Unknown,Rhetoric/Composition,Wiki,mailto:as_dean@aub.edu.lb | http://www.aub.edu...
19,Cal State San Marcos,2011,http://www.csusm.edu/facultyopportunities,Unknown,Rhetoric/Composition,Wiki,http://www.csusm.edu/facultyopportunities
26,Claremont McKenna College,2011,https://webapps.cmc.edu/jobs/staff/staff_openi...,Unknown,Rhetoric/Composition,Wiki,https://webapps.cmc.edu/jobs/staff/staff_openi...
34,Columbia University (NY),2011,http://chronicle.com/jobs/0000649408-01/ | htt...,Unknown,Rhetoric/Composition,Wiki,http://chronicle.com/jobs/0000649408-01/ | htt...
36,Concordia University Chicago,2011,http://www.higheredjobs.com/faculty/details.cf...,Unknown,Rhetoric/Composition,Wiki,http://www.higheredjobs.com/faculty/details.cf...
...,...,...,...,...,...,...,...
9235,Senior Faculty Positions,2020,,Unknown,German,Wiki,
9241,Faculty Positions outside of Canada and the U.S.,2020,https://www.lalia-berlin.com/jobs-1-1,Unknown,German,Wiki,https://www.lalia-berlin.com/jobs-1-1
9242,Multi-Language Faculty Positions,2020,https://aprecruit.berkeley.edu/JPF02679,Unknown,German,Wiki,https://aprecruit.berkeley.edu/JPF02679
9244,Other German-Related Faculty Positions,2020,,Unknown,German,Wiki,


In [26]:
list(set(df_all.JobField.values.tolist()))

['Children%27s',
 'TESOL / Applied Linguistics',
 'Drama, Speech, Film %26 New Media',
 'English Literature',
 'Modern British',
 'Medieval',
 'Restoration/18th-Century',
 'Environmental 2014#Christopher Newport University - Lecturer in English .28Environmental Rhetoric .26 Science Writing.29',
 'American Open',
 'African American',
 'American Studies',
 'Comparative Literature',
 'New Media and Digital Humanities',
 'Musicology/Ethnomusicology',
 'New Media %26 Digital Humanities',
 'Theory',
 'Renaissance',
 'Communication and Media Studies',
 'Music Theory/Composition',
 'Generalist',
 'Environmental',
 'Queer/Women%27s/Gender Studies 2014#Texas A.26M - TT Advanced Assistant Professor.2C Gender and Sexuality Studies with an Emphasis in',
 'Comparative',
 'Other Ethnic American',
 '20-21 c. American',
 'Spanish and Portuguese',
 'Creative Writing',
 'Queer/Women%27s/Gender Studies',
 'Rhetoric/Composition',
 'Romanticism/Victorian/19th',
 'Humanities and Social Sciences Postdocs',
 '

In [27]:
len(df_all)

7348

In [28]:
df_all.loc[(df_all.JobField == 'New Media %26 Digital Humanities'), ('JobField')] = 'New Media and Digital Humanities'
df_all.loc[(df_all.JobField == 'Queer/Women%27s/Gender Studies'), ('JobField')] = 'Queer/Womens/Gender Studies'
df_all.loc[(df_all.JobField == 'Queer/Women%27s/Gender Studies 2014#Texas A.26M - TT Advanced Assistant Professor.2C Gender and Sexuality Studies with an Emphasis in'), ('JobField')] = 'Queer/Womens/Gender Studies'
df_all.loc[(df_all.JobField == 'TESOL / Applied Linguistics'), ('JobField')] = 'TESOL/Applied Linguistics'
df_all.loc[(df_all.JobField == 'Drama, Speech, Film %26 New Media'), ('JobField')] = 'Drama, Speech, Film and New Media'
df_all.loc[(df_all.JobField == 'Environmental 2014#Christopher Newport University - Lecturer in English .28Environmental Rhetoric .26 Science Writing.29'), ('JobField')] = 'Environmental'
len(df_all)

7348

In [29]:
list(set(df_all.JobField.values.tolist()))

['Children%27s',
 'English Literature',
 'Modern British',
 'Medieval',
 'Restoration/18th-Century',
 'American Open',
 'African American',
 'American Studies',
 'Comparative Literature',
 'New Media and Digital Humanities',
 'Drama, Speech, Film and New Media',
 'Musicology/Ethnomusicology',
 'Theory',
 'Renaissance',
 'Communication and Media Studies',
 'Music Theory/Composition',
 'Generalist',
 'Environmental',
 'Queer/Womens/Gender Studies',
 'Comparative',
 'Other Ethnic American',
 '20-21 c. American',
 'Spanish and Portuguese',
 'Creative Writing',
 'Rhetoric/Composition',
 'Romanticism/Victorian/19th',
 'Humanities and Social Sciences Postdocs',
 'Community Colleges',
 'Anglophone',
 'TESOL/Applied Linguistics',
 'CreativeWriting',
 'German',
 'Social Work',
 'British Open']

In [30]:
field_list = [
 'Communication and Media Studies',
 'Rhetoric/Composition',
 'Drama, Speech, Film and New Media',
 'Queer/Womens/Gender Studies',
 'Generalist',
 'New Media and Digital Humanities',
 'TESOL/Applied Linguistics',
 'Humanities and Social Sciences Postdocs',
 'Community Colleges'
]

In [31]:
df_reduced = df_all[df_all.JobField.isin(field_list)]
len(df_reduced)

4468

## Save field names

In [32]:
# Download names
field_alias_fn = 'data.jobcensus.rc.field_aliases.csv'
if not os.path.exists(field_alias_fn):
    df_aliases=df_reduced.JobField.value_counts().reset_index()
    df_aliases.columns=['JobField','Count']
    df_aliases['FieldAlias']=df_aliases['JobField']
    df_aliases[['JobField','FieldAlias','Count']].sort_values('JobField').to_csv(field_alias_fn,index=False)

In [33]:
# Edit them!

# Save

In [34]:
now=datetime.now()
date=f'{now.year}-{now.month}-{now.day}'
ofn=f'data.jobcensus.rc.synthesized.{date}.csv'
df_reduced.to_csv(ofn,index=False)